# Gesture Recognition
In this project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

C:\Users\ragarwal\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 50 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [0,2,6,9,12,15,18,21,24,27] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_sequences = len(folder_list)
        num_batches = int(num_sequences/batch_size) # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,10,120,120,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    if(image.shape[0] == 360 and image.shape[1] == 360): # Resize 360X360 image to 120X120
                        image = imresize(image, (120, 120))
                    else:
                        image = image[:,20:140] # Crop the 120X160 image to 120X120
                    
                    batch_data[folder,idx] = image
                    
                    batch_data[folder,idx,:,:,0] = batch_data[folder,idx,:,:,0] - (np.min(batch_data[folder,idx,:,:,0])/
                                                    np.max(batch_data[folder,idx,:,:,0]) - np.min(batch_data[folder,idx,:,:,0]))
                    batch_data[folder,idx,:,:,1] = batch_data[folder,idx,:,:,1] - (np.min(batch_data[folder,idx,:,:,1])/
                                                    np.max(batch_data[folder,idx,:,:,1]) - np.min(batch_data[folder,idx,:,:,1]))
                    batch_data[folder,idx,:,:,2] = batch_data[folder,idx,:,:,2] - (np.min(batch_data[folder,idx,:,:,2])/
                                                    np.max(batch_data[folder,idx,:,:,2]) - np.min(batch_data[folder,idx,:,:,2]))
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        remaining_steps = num_sequences%batch_size
        
        if (remaining_steps) > 0:
            batch_data = np.zeros((remaining_steps,10,120,120,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((remaining_steps,5)) # batch_labels is the one hot representation of the output
            for folder in range(remaining_steps): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    if(image.shape[0] == 360 and image.shape[1] == 360): # Resize 360X360 image to 120X120
                        image = imresize(image, (120, 120))
                    else:
                        image = image[:,20:140] # Crop the 120X160 image to 120X120
                    
                    batch_data[folder,idx] = image
                    
                    batch_data[folder,idx,:,:,0] = batch_data[folder,idx,:,:,0] - (np.min(batch_data[folder,idx,:,:,0])/
                                                    np.max(batch_data[folder,idx,:,:,0]) - np.min(batch_data[folder,idx,:,:,0]))
                    batch_data[folder,idx,:,:,1] = batch_data[folder,idx,:,:,1] - (np.min(batch_data[folder,idx,:,:,1])/
                                                    np.max(batch_data[folder,idx,:,:,1]) - np.min(batch_data[folder,idx,:,:,1]))
                    batch_data[folder,idx,:,:,2] = batch_data[folder,idx,:,:,2] - (np.min(batch_data[folder,idx,:,:,2])/
                                                    np.max(batch_data[folder,idx,:,:,2]) - np.min(batch_data[folder,idx,:,:,2]))
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
            

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [5]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 100 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 100


## Model
### 3D Convolutional Model

In [6]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here

model = Sequential()
model.add(Conv3D(32, (3, 3, 3), padding='same',
                 input_shape=(10,120,120,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv3D(32, (3, 3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.25))

model.add(Conv3D(64, (3, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv3D(64, (3, 3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512,kernel_regularizer=l2(0.01)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [7]:
optimiser = optimizers.SGD()#write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 10, 120, 120, 32)  2624      
_________________________________________________________________
activation_1 (Activation)    (None, 10, 120, 120, 32)  0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 10, 120, 120, 32)  128       
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 8, 118, 118, 32)   27680     
_________________________________________________________________
activation_2 (Activation)    (None, 8, 118, 118, 32)   0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 8, 118, 118, 32)   128       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 4, 59, 59, 32)     0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [8]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [9]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001)# write the REducelronplateau code here

callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [10]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [12]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/valSource path =  ./Project_data/train ; batch size =Epoch 1/100
 ; batch size = 50
 50


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


12/14 [========================>.....] - ETA: 9s - loss: 21.2817 - categorical_accuracy: 0.2167 

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:52: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


14/14 [==============================] - 70s 5s/step - loss: 20.7315 - categorical_accuracy: 0.2384 - val_loss: 22.5267 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to model_init_2019-06-2305_11_09.891400/model-00001-20.96782-0.23077-22.52668-0.23000.h5
Epoch 2/100
14/14 [==============================] - 37s 3s/step - loss: 14.5030 - categorical_accuracy: 0.4191 - val_loss: 18.3263 - val_categorical_accuracy: 0.2400

Epoch 00002: saving model to model_init_2019-06-2305_11_09.891400/model-00002-14.42102-0.43288-18.32626-0.24000.h5
Epoch 3/100
14/14 [==============================] - 37s 3s/step - loss: 12.5403 - categorical_accuracy: 0.4927 - val_loss: 22.4436 - val_categorical_accuracy: 0.2300

Epoch 00003: saving model to model_init_2019-06-2305_11_09.891400/model-00003-12.40685-0.50226-22.44357-0.23000.h5
Epoch 4/100
14/14 [==============================] - 37s 3s/step - loss: 12.0597 - categorical_accuracy: 0.5773 - val_loss: 11.2851 - val_categorical_accuracy: 0.5

Epoch 30/100
14/14 [==============================] - 37s 3s/step - loss: 10.0635 - categorical_accuracy: 0.8994 - val_loss: 10.6678 - val_categorical_accuracy: 0.6700

Epoch 00030: saving model to model_init_2019-06-2305_11_09.891400/model-00030-9.94779-0.92609-10.66784-0.67000.h5
Epoch 31/100
14/14 [==============================] - 37s 3s/step - loss: 10.1132 - categorical_accuracy: 0.8861 - val_loss: 10.6091 - val_categorical_accuracy: 0.6700

Epoch 00031: saving model to model_init_2019-06-2305_11_09.891400/model-00031-9.96793-0.92006-10.60908-0.67000.h5
Epoch 32/100
14/14 [==============================] - 37s 3s/step - loss: 10.1522 - categorical_accuracy: 0.8884 - val_loss: 10.5953 - val_categorical_accuracy: 0.6900

Epoch 00032: saving model to model_init_2019-06-2305_11_09.891400/model-00032-9.95915-0.91855-10.59525-0.69000.h5
Epoch 33/100
14/14 [==============================] - 37s 3s/step - loss: 10.2376 - categorical_accuracy: 0.8823 - val_loss: 10.5869 - val_categorical_

Epoch 59/100
14/14 [==============================] - 37s 3s/step - loss: 10.0529 - categorical_accuracy: 0.9090 - val_loss: 10.5355 - val_categorical_accuracy: 0.6400

Epoch 00059: saving model to model_init_2019-06-2305_11_09.891400/model-00059-9.78573-0.94419-10.53545-0.64000.h5
Epoch 60/100
14/14 [==============================] - 37s 3s/step - loss: 9.9338 - categorical_accuracy: 0.9004 - val_loss: 10.5514 - val_categorical_accuracy: 0.6300

Epoch 00060: saving model to model_init_2019-06-2305_11_09.891400/model-00060-9.74449-0.93514-10.55137-0.63000.h5
Epoch 61/100
14/14 [==============================] - 37s 3s/step - loss: 10.0401 - categorical_accuracy: 0.8937 - val_loss: 10.5047 - val_categorical_accuracy: 0.6100

Epoch 00061: saving model to model_init_2019-06-2305_11_09.891400/model-00061-9.77403-0.93213-10.50466-0.61000.h5
Epoch 62/100
14/14 [==============================] - 37s 3s/step - loss: 10.0188 - categorical_accuracy: 0.8899 - val_loss: 10.5351 - val_categorical_a

14/14 [==============================] - 37s 3s/step - loss: 9.8797 - categorical_accuracy: 0.9114 - val_loss: 10.2812 - val_categorical_accuracy: 0.7000

Epoch 00088: saving model to model_init_2019-06-2305_11_09.891400/model-00088-9.57995-0.95475-10.28125-0.70000.h5
Epoch 89/100
14/14 [==============================] - 37s 3s/step - loss: 9.7876 - categorical_accuracy: 0.9152 - val_loss: 10.2849 - val_categorical_accuracy: 0.6900

Epoch 00089: saving model to model_init_2019-06-2305_11_09.891400/model-00089-9.56089-0.95475-10.28494-0.69000.h5
Epoch 90/100
14/14 [==============================] - 37s 3s/step - loss: 9.6796 - categorical_accuracy: 0.9314 - val_loss: 10.2819 - val_categorical_accuracy: 0.7000

Epoch 00090: saving model to model_init_2019-06-2305_11_09.891400/model-00090-9.52579-0.96380-10.28187-0.70000.h5
Epoch 91/100
14/14 [==============================] - 37s 3s/step - loss: 9.7106 - categorical_accuracy: 0.9333 - val_loss: 10.2775 - val_categorical_accuracy: 0.6900


### The best accuracy values of the Conv3D model were received in Epoch# 99
#### Training accuracy --> 0.95928
#### Validation accuracy --> 0.7200

## CNN - RNN Model

### We are using ResNet50 model layers to leverage transfer learning in 2DCNN layers. These layers are stacked with a GRU layer to complete the CNN-RNN architecture. 

In [11]:
batch_size = 40

## Generator

### We need to write a different generator function (generator_2) as the input image size for ResNet50 is (224, 224, 3)



In [13]:
def generator_2(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [0,2,6,9,12,15,18,21,24,27] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_sequences = len(folder_list)
        num_batches = int(num_sequences/batch_size) # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,10,224,224,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    if(image.shape[0] == 360 and image.shape[1] == 360): # Resize the 360X360 image to 224X224
                        image = imresize(image, (224, 224)) 
                    else:
                        image = image[:,20:140] # First crop the 120X160 image to 120X120
                        image = imresize(image, (224, 224)) # Now resize the 120X120 image to 224X224
                    
                    batch_data[folder,idx] = image
                    
                    batch_data[folder,idx,:,:,0] = batch_data[folder,idx,:,:,0] - (np.min(batch_data[folder,idx,:,:,0])/
                                                    np.max(batch_data[folder,idx,:,:,0]) - np.min(batch_data[folder,idx,:,:,0]))
                    batch_data[folder,idx,:,:,1] = batch_data[folder,idx,:,:,1] - (np.min(batch_data[folder,idx,:,:,1])/
                                                    np.max(batch_data[folder,idx,:,:,1]) - np.min(batch_data[folder,idx,:,:,1]))
                    batch_data[folder,idx,:,:,2] = batch_data[folder,idx,:,:,2] - (np.min(batch_data[folder,idx,:,:,2])/
                                                    np.max(batch_data[folder,idx,:,:,2]) - np.min(batch_data[folder,idx,:,:,2]))
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        remaining_steps = num_sequences%batch_size
        
        if (remaining_steps) > 0:
            batch_data = np.zeros((remaining_steps,10,224,224,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((remaining_steps,5)) # batch_labels is the one hot representation of the output
            for folder in range(remaining_steps): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    if(image.shape[0] == 360 and image.shape[1] == 360): # Resize the 360X360 image to 224X224
                        image = imresize(image, (224, 224))
                    else:
                        image = image[:,20:140] # First crop the 120X160 image to 120X120
                        image = imresize(image, (224, 224)) # Now resize the 120X120 image to 224X224
                    
                    batch_data[folder,idx] = image
                    
                    batch_data[folder,idx,:,:,0] = batch_data[folder,idx,:,:,0] - (np.min(batch_data[folder,idx,:,:,0])/
                                                    np.max(batch_data[folder,idx,:,:,0]) - np.min(batch_data[folder,idx,:,:,0]))
                    batch_data[folder,idx,:,:,1] = batch_data[folder,idx,:,:,1] - (np.min(batch_data[folder,idx,:,:,1])/
                                                    np.max(batch_data[folder,idx,:,:,1]) - np.min(batch_data[folder,idx,:,:,1]))
                    batch_data[folder,idx,:,:,2] = batch_data[folder,idx,:,:,2] - (np.min(batch_data[folder,idx,:,:,2])/
                                                    np.max(batch_data[folder,idx,:,:,2]) - np.min(batch_data[folder,idx,:,:,2]))
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

In [14]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 100 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 100


## Model


In [15]:
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense, Dropout, Flatten
from keras.models import Model
from keras.layers import TimeDistributed
from keras.layers import GRU
from keras.layers import Input, Reshape

#write your model here

# Get base model: ResNet50 
base_model = ResNet50(weights='imagenet', include_top=False)

# training the model after 140 layers. Freezing first 140 layers
split_at = 140
for layer in base_model.layers[:split_at]: layer.trainable = False
for layer in base_model.layers[split_at:]: layer.trainable = True

# Get the output from the base model 
base_model_ouput = base_model.output

# Adding our own layers at the end
# global average pooling: computes the average of all values in the feature map
x = GlobalAveragePooling2D()(base_model_ouput)

# fully connected layer
x = Dense(512, activation='relu', kernel_regularizer=l2(0.01))(x)

resnet_model = Model(inputs=base_model.input, outputs=x)

In [16]:
frames = Input(shape=(10, 224, 224, 3))
x = TimeDistributed(resnet_model)(frames)
x = TimeDistributed(Flatten())(x)

x = GRU(64, return_sequences=False)(x)

x = Dense(5, activation='softmax')(x)

gru_model = Model(inputs=frames, outputs=x)


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [17]:
optimiser = optimizers.SGD()#write your optimizer
gru_model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (gru_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 10, 224, 224, 3)   0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 512)           24636800  
_________________________________________________________________
time_distributed_2 (TimeDist (None, 10, 512)           0         
_________________________________________________________________
gru_1 (GRU)                  (None, 64)                110784    
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 325       
Total params: 24,747,909
Trainable params: 16,136,197
Non-trainable params: 8,611,712
_________________________________________________________________
None


Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [18]:
train_generator = generator_2(train_path, train_doc, batch_size)
val_generator = generator_2(val_path, val_doc, batch_size)

In [19]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001)# write the REducelronplateau code here

callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [20]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [20]:
gru_model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 40
Source path =  ./Project_data/train ; batch size = 40
Epoch 1/100


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launc

17/17 [==============================] - 137s 8s/step - loss: 9.7056 - categorical_accuracy: 0.3855 - val_loss: 9.7585 - val_categorical_accuracy: 0.2900

Epoch 00001: saving model to model_init_2019-06-2318_25_41.949303/model-00001-9.70349-0.39065-9.75852-0.29000.h5
Epoch 2/100
17/17 [==============================] - 80s 5s/step - loss: 9.0930 - categorical_accuracy: 0.6505 - val_loss: 9.5111 - val_categorical_accuracy: 0.4200

Epoch 00002: saving model to model_init_2019-06-2318_25_41.949303/model-00002-9.05362-0.66516-9.51111-0.42000.h5
Epoch 3/100
17/17 [==============================] - 76s 4s/step - loss: 8.7882 - categorical_accuracy: 0.7923 - val_loss: 9.4291 - val_categorical_accuracy: 0.4800

Epoch 00003: saving model to model_init_2019-06-2318_25_41.949303/model-00003-8.73286-0.81146-9.42912-0.48000.h5
Epoch 4/100
17/17 [==============================] - 76s 4s/step - loss: 8.6311 - categorical_accuracy: 0.8425 - val_loss: 9.8427 - val_categorical_accuracy: 0.1700

Epoch 00

17/17 [==============================] - 75s 4s/step - loss: 6.0595 - categorical_accuracy: 0.9575 - val_loss: 6.5569 - val_categorical_accuracy: 0.8000

Epoch 00059: saving model to model_init_2019-06-2318_25_41.949303/model-00059-5.98961-0.97436-6.55691-0.80000.h5
Epoch 60/100
17/17 [==============================] - 75s 4s/step - loss: 6.0344 - categorical_accuracy: 0.9600 - val_loss: 6.6072 - val_categorical_accuracy: 0.7900

Epoch 00060: saving model to model_init_2019-06-2318_25_41.949303/model-00060-5.97105-0.97587-6.60721-0.79000.h5
Epoch 61/100
17/17 [==============================] - 75s 4s/step - loss: 6.0421 - categorical_accuracy: 0.9575 - val_loss: 6.6698 - val_categorical_accuracy: 0.7700

Epoch 00061: saving model to model_init_2019-06-2318_25_41.949303/model-00061-5.97143-0.97436-6.66981-0.77000.h5
Epoch 62/100
17/17 [==============================] - 75s 4s/step - loss: 6.0290 - categorical_accuracy: 0.9500 - val_loss: 6.7083 - val_categorical_accuracy: 0.7400

Epoch 

### The best accuracy values of the CNN-RNN model were received in Epoch# 100
#### Training accuracy --> 0.97587
#### Validation accuracy --> 0.7500

In [21]:
weights_path = './model-00100-5.79901-0.97587-6.55195-0.75000.h5'
gru_model.load_weights(weights_path)

In [22]:
print(gru_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 10, 224, 224, 3)   0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 512)           24636800  
_________________________________________________________________
time_distributed_2 (TimeDist (None, 10, 512)           0         
_________________________________________________________________
gru_1 (GRU)                  (None, 64)                110784    
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 325       
Total params: 24,747,909
Trainable params: 16,136,197
Non-trainable params: 8,611,712
_________________________________________________________________
None
